In [1]:
import pandas as pd
import numpy as np
import sys
import gzip
import pickle
import pandas as pd

from importlib import reload

#provide the paths to the packages ecospold2matrix, pymrio and pylcaio
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/ecospold2matrix/')
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/pymrio/')
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/pylcaio/src/')
import ecospold2matrix as e2m
import pymrio
import pylcaio



## Prepare Ecoinvent data with ecospold2matrix

In [2]:
%%time

#run ecospold2matrix
ecospold_dir = '/home/jakobs/Documents/IndEcol/Data/ecoinvent/ecoinvent 3.5_cutoff_ecoSpold02/'
project_name = 'ecoinvent_35_cutoff'
out_dir = '/home/jakobs/Documents/IndEcol/Data/ecospold2matrix_matrices/'

parser = e2m.Ecospold2Matrix(ecospold_dir, project_name, out_dir, positive_waste=True, nan2null=True,
                            characterisation_file=
        '/home/jakobs/data/ecoinvent/ecoinvent 3.5_LCIA_implementation/LCIA_implementation_3.5.xlsx', )
parser.ecospold_to_Leontief(fileformats='Pandas',with_absolute_flows=False)

2020-12-01 11:19:33,018 - ecoinvent_35_cutoff - INFO - Ecospold2Matrix Processing
2020-12-01 11:19:33,032 - ecoinvent_35_cutoff - INFO - Current git commit: 5e2fd295217611ce579ea47865d975d52d229058
2020-12-01 11:19:33,033 - ecoinvent_35_cutoff - INFO - Project name: ecoinvent_35_cutoff
2020-12-01 11:19:33,034 - ecoinvent_35_cutoff - INFO - Unit process and Master data directory: /home/jakobs/Documents/IndEcol/Data/ecoinvent/ecoinvent 3.5_cutoff_ecoSpold02/
2020-12-01 11:19:33,035 - ecoinvent_35_cutoff - INFO - Data saved in: /home/jakobs/Documents/IndEcol/Data/ecospold2matrix_matrices
2020-12-01 11:19:33,036 - ecoinvent_35_cutoff - INFO - Sign conventions changed to make waste flows positive
2020-12-01 11:19:33,037 - ecoinvent_35_cutoff - INFO - Replace Not-a-Number instances with 0.0 in all matrices
2020-12-01 11:19:33,038 - ecoinvent_35_cutoff - INFO - Pickle intermediate results to files
2020-12-01 11:19:33,038 - ecoinvent_35_cutoff - INFO - Order processes based on: ISIC, activityN

starting characterisation


2020-12-01 11:25:31,298 - ecoinvent_35_cutoff - INFO - Will use column 7, named CF_35, for characterisation factors
2020-12-01 11:25:31,299 - ecoinvent_35_cutoff - INFO - Starting characterisation matching
2020-12-01 11:25:39,810 - ecoinvent_35_cutoff - INFO - Characterisation matching done. C matrix created
2020-12-01 11:25:39,885 - ecoinvent_35_cutoff - INFO - Starting to export to file
2020-12-01 11:25:39,886 - ecoinvent_35_cutoff - INFO - about to write to file
2020-12-01 11:26:45,982 - ecoinvent_35_cutoff - INFO - Final, symmetric, normalized matrices saved in /home/jakobs/Documents/IndEcol/Data/ecospold2matrix_matrices/ecoinvent_35_cutoffPandas_symmNorm.gz.pickle with SHA-1 of cecb21df9cd4c91641be7c11d2195b2429f93ac5
2020-12-01 11:26:45,983 - ecoinvent_35_cutoff - INFO - Done running ecospold2matrix.ecospold_to_Leontief


CPU times: user 7min 1s, sys: 7.98 s, total: 7min 9s
Wall time: 7min 12s


## Load ecoinvent pickle files

In [2]:
%%time
with gzip.open('/home/jakobs/Documents/IndEcol/Data/ecospold2matrix_matrices/ecoinvent_35_cutoffPandas_symmNorm.gz.pickle','rb') as f:
    ecoinvent = pd.read_pickle(f)

CPU times: user 7.63 s, sys: 423 ms, total: 8.05 s
Wall time: 8.07 s


### Imput missing ecoinvent price data
Ecospold2matrix seems to have a bug and not extract all price data. I do not have the time to fix this at the moment. However the datasheets in filter.xlsx contains the actual ecoinvent prices for the processes so I will just manually imput them into the PRO dataframe

In [3]:
file =  pd.ExcelFile('/home/jakobs/Documents/IndEcol/OASES/pylcaio/src/Data/eco3.5_exio3/Filter.xlsx')
sheetnames = file.sheet_names

for sheet_name in sheetnames:
    if not sheet_name == 'No price':
        print(sheet_name)
        sheet = file.parse(sheet_name, index_col=0)
        #find UUID of those processes that do not have a price (NaN)
        indices = list(set(ecoinvent['PRO'].loc[ecoinvent['PRO'].price.isnull()].index.tolist()).intersection(
                  sheet.index.tolist()))
        ecoinvent['PRO'].loc[indices, 'price'] = sheet.loc[indices, 'Price']
        ecoinvent['PRO'].loc[indices, 'priceUnit'] = 'EUR2005'

Hybridized
Market
Not commercialized
Poor quality
Empty processes
Uncovered geography


## Load Exiobase into a Dataframe with pymrio

In [4]:
#EXIOBASE 3_8 is not good! A matrix for 2014 contains many negatives!

In [4]:
%%time
# io = pymrio.parse_exiobase3('/home/jakobs/data/EXIOBASE/exiobase3_6/IOT_2014_pxp/')
io = pymrio.parse_exiobase3('/home/jakobs/data/EXIOBASE/exiobase3_6/IOT_2012_pxp/')

CPU times: user 32.1 s, sys: 648 ms, total: 32.7 s
Wall time: 32.7 s


In [5]:
(io.A <0).any().any()

False

## Read in data with pylcaio

In [5]:
# reload(pylcaio)

In [5]:
%%time
database_loader = pylcaio.DatabaseLoader(ecoinvent, io, 'ecoinvent3.5', 'exiobase3')

CPU times: user 10.7 ms, sys: 79 µs, total: 10.8 ms
Wall time: 5.39 ms


In [6]:
%%time
lcaio_object = database_loader.combine_ecoinvent_exiobase(complete_extensions=False, 
                                        impact_world=False, regionalized=False)

No path for the capital folder was provided. Capitals will not be endogenized
CPU times: user 2min 38s, sys: 4.65 s, total: 2min 43s
Wall time: 1min 39s


In [7]:
%%time
lcaio_object.hybridize(price_neutral_cut_off_matrix=True, capitals=False)

CPU times: user 12min 32s, sys: 1min 4s, total: 13min 37s
Wall time: 8min 24s


In [8]:
%%time
lcaio_object.correct_double_counting('STAM', capitals=False)

CPU times: user 6min 7s, sys: 9.22 s, total: 6min 17s
Wall time: 4min 22s


In [8]:
%%time
lcaio_object.correct_double_counting('binary', capitals=False)

CPU times: user 2min 16s, sys: 4.56 s, total: 2min 20s
Wall time: 1min 23s


In [9]:
%%time
lcaio_object.save_system(format='pickle')

CPU times: user 24.4 s, sys: 288 ms, total: 24.7 s
Wall time: 24.6 s
